In [ ]:
import os
from ydata_profiling import ProfileReport
from ydata_profiling.config import Settings
import argparse
import pandas as pd
import hashlib
import json
import getpass
import oracledb
from dotenv import load_dotenv
#from .autonotebook import tqdm as notebook_tqdm

In [ ]:
def connect_to_oracle(db_user: str, db_password: str, db_host: str, db_port, db_sid) -> None:
    try: 
        with oracledb.connect(user=db_user, password=db_password, dsn=oracledb.makedsn(db_host, db_port, db_sid)) as connection:
            with connection.cursor() as cursor:
                sql = """select sysdate from dual"""
                for r in cursor.execute(sql):
                    print(r)
    except oracledb.DatabaseError as e:
        error, = e.args
        print(f"Oracle error code: {error.code}")
        print(f"Oracle error message: {error.message}")
    return None

def get_list_of_tables(path_to_file: str) -> list:
    """
    Get a list of tables from a file.
    
    Args:
        path_to_file (str): path to file containing table names
        
    Returns:
        list: List of table names
    """
    try:
        with open(path_to_file, 'r') as file:
            tables = [line.strip() for line in file.readlines()]
        return tables
    except FileNotFoundError:
        print(f"Error: File not found at the path '{path_to_file}'. Please check the path and try again.")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []

def hash_column(column: pd.Series) -> pd.Series:
    """Hash the values in a column using SHA-256."""
    return column.apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest() if pd.notnull(x) else x)

def generate_profiling_report(db_connection=None, schema=None, tables_list=None, path_to_csv=None, 
                             sensitive_columns=None, sensitive_keywords=None) -> pd.DataFrame:
    """
    Generate a profiling report using ydata-profiling for either CSV files or Oracle database tables
    
    Args:
        db_connection (oracledb.Connection, optional): Connection to Oracle database
        schema (str, optional): Schema name for Oracle tables
        tables_list (list, optional): List of tables to profile
        path_to_csv (str, optional): Path to CSV file (if not using database)
        sensitive_columns (list, optional): List of column names to mark as sensitive
        sensitive_keywords (list, optional): Keywords to detect sensitive columns
        
    Returns:
        DataFrame: DataFrame with profiling information including completeness metrics
    """
    # Default sensitive keywords if not provided
    if sensitive_keywords is None:
        sensitive_keywords = ["bvn", "id number", "nin", "passport", "driver", 
                             "identificationnumber", "chn"]
    
    results_dfs = []  # To store results from multiple tables
    
    try:
        # Determine source type (CSV or database)
        if path_to_csv:
            # Process CSV file
            file_name = str.split(path_to_csv, "/")[-1].split(".")[0]
            data = pd.read_csv(path_to_csv)
            source_name = file_name
            table_name = file_name
            schema_name = None
            
            # Process single CSV
            result_df = _process_dataset(data, source_name, table_name, schema_name, 
                                         sensitive_columns, sensitive_keywords)
            results_dfs.append(result_df)
            
        elif db_connection and tables_list:
            # Process Oracle tables
            for table in tables_list:
                try:
                    print(f"Processing table: {schema}.{table}")
                    query = f"SELECT * FROM {schema}.{table}"
                    data = pd.read_sql(query, db_connection)
                    
                    # For empty tables
                    if data.empty:
                        print(f"Table {schema}.{table} is empty. Skipping.")
                        continue
                    
                    result_df = _process_dataset(data, f"{schema}.{table}", table, schema,
                                               sensitive_columns, sensitive_keywords)
                    results_dfs.append(result_df)
                    
                except Exception as e:
                    print(f"Error processing table {schema}.{table}: {e}")
        else:
            print("Error: Either provide path_to_csv or both db_connection and tables_list")
            return pd.DataFrame()
        
        # Combine all results
        if results_dfs:
            combined_df = pd.concat(results_dfs)
            return combined_df
        else:
            return pd.DataFrame()
        
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return pd.DataFrame()

def _process_dataset(data, source_name, table_name, schema_name, sensitive_columns, sensitive_keywords):
    """Helper function to process a single dataset (CSV or DB table)"""
    # Store the total record count
    total_records = len(data)
    
    # Automatically detect sensitive columns if not provided
    if sensitive_columns is None:
        sensitive_columns = [
            col for col in data.columns 
            if any(keyword in col.lower() for keyword in sensitive_keywords)
        ]
    
    # Hash sensitive columns
    if sensitive_columns:
        for col in sensitive_columns:
            if col in data.columns:
                print(f"Hashing sensitive column: {col}")
                data[col] = hash_column(data[col])
    
    # Configure settings to mark sensitive columns
    config = Settings()
    if sensitive_columns:
        config.variables.descriptions = {col: "Sensitive Data (Hashed)" 
                                       for col in sensitive_columns 
                                       if col in data.columns}
    
    # Generate profiling report    
    profile = ProfileReport(
        data,
        title=f"{source_name} Profiling Report",
        explorative=True,
        config=config)
    
    # Get JSON data and extract variables data
    json_data = profile.to_json()
    variables_data = json.loads(json_data)['variables']
    variables_df = pd.DataFrame(variables_data).transpose()
    variables_df = variables_df.reset_index().rename(columns={'index': 'column_name'})
    
    # Add metadata enrichment
    variables_df['table_name'] = table_name
    variables_df['schema_name'] = schema_name
    variables_df['total_records'] = total_records
    variables_df['created_at'] = pd.Timestamp.now()
    variables_df['last_updated'] = pd.Timestamp.now()
    
    # Calculate completeness percentage
    if 'count' in variables_df.columns and 'n_missing' in variables_df.columns:
        variables_df['completeness_pct'] = ((variables_df['count'] - variables_df['n_missing']) / 
                                         variables_df['count'] * 100).round(2)
    
    # Mark sensitive columns in the metadata
    variables_df['is_sensitive'] = variables_df['column_name'].isin(sensitive_columns)
    
    print(f"Successfully generated profile for {source_name} with {len(variables_df)} columns")
    
    return variables_df

def generate_metadata_file(var_df: pd.DataFrame, output_path: str = None) -> pd.DataFrame:
    """
    Generate a metadata file from the profiling report.
    If the file exists, checks for duplicates and appends only new records.
    
    Args:
        var_df (DataFrame): DataFrame with profiling information
        output_path (str, optional): Path to save the metadata file
    
    Returns:
        DataFrame: The processed metadata DataFrame that was saved
    """
    try:
        # Create a copy to avoid modifying the original DataFrame
        metadata_df = var_df.copy()
        
        # Set default filename based on schema and table if available
        if output_path is None:
            if 'schema_name' in metadata_df.columns and 'table_name' in metadata_df.columns:
                # Use first row's schema and table (assuming all rows are for same table)
                schema = metadata_df['schema_name'].iloc[0]
                table = metadata_df['table_name'].iloc[0]
                if schema and table:
                    output_path = f"{schema}_{table}_metadata.csv"
                else:
                    output_path = "metadata_report.csv"
            else:
                output_path = "metadata_report.csv"
        
        # Rename columns if they have their original names
        rename_map = {
            'n_distinct': 'distinct_count',
            'p_distinct': 'distinct_percentage',
            'is_unique': 'is_unique',
            'type': 'data_type',
            'n_unique': 'unique_count',
            'p_unique': 'unique_percentage',
            'n_missing': 'missing_count',
            'n': 'total_count',
            'p_missing': 'missing_percentage',
            'n_category': 'category_count'
        }
        
        # Only rename columns that exist and haven't been renamed yet
        rename_cols = {k: v for k, v in rename_map.items() if k in metadata_df.columns and v not in metadata_df.columns}
        if rename_cols:
            metadata_df.rename(columns=rename_cols, inplace=True)
        
        # Ensure these columns are included at the beginning
        priority_cols = ['schema_name', 'table_name', 'column_name', 'data_type', 
                        'total_records', 'total_count', 'missing_count', 
                        'completeness_pct', 'is_sensitive',
                        'created_at', 'last_updated']
        
        # Create a list of all columns with priority columns first
        all_cols = []
        for col in priority_cols:
            if col in metadata_df.columns:
                all_cols.append(col)
                
        # Add remaining columns
        for col in metadata_df.columns:
            if col not in all_cols:
                all_cols.append(col)
                
        # Reorder columns
        metadata_df = metadata_df[all_cols]
        
        # Check if file exists and handle append logic
        if os.path.exists(output_path):
            # Read existing metadata
            existing_df = pd.read_csv(output_path)
            
            # Define key columns for identifying duplicates
            # A row is considered a duplicate if schema, table, and column name match
            key_columns = ['schema_name', 'table_name', 'column_name']
            key_columns = [col for col in key_columns if col in metadata_df.columns and col in existing_df.columns]
            
            if key_columns:  # Only proceed with duplicate check if we have key columns
                # Filter out rows that already exist in the file
                # Create a set of tuples with the key values from existing data
                existing_keys = set(
                    tuple(row) for row in existing_df[key_columns].itertuples(index=False, name=None)
                )
                
                # Filter new data to only include rows with new keys
                new_data_mask = ~metadata_df.apply(
                    lambda row: tuple(row[key_columns]) in existing_keys, axis=1
                )
                
                # If we have any new data, append it
                if new_data_mask.any():
                    metadata_df = metadata_df[new_data_mask]
                    # Append new data to existing file
                    metadata_df.to_csv(output_path, mode='a', header=False, index=False)
                    print(f"Appended {new_data_mask.sum()} new records to {output_path}")
                else:
                    print(f"No new records to append to {output_path}")
                    
                # Combine for return value
                metadata_df = pd.concat([existing_df, metadata_df])
            else:
                # If we can't determine duplicates, append all (might cause duplicates)
                metadata_df.to_csv(output_path, mode='a', header=False, index=False)
                print(f"Appended all records to {output_path} (duplicate checking unavailable)")
                
        else:
            # File doesn't exist, create new
            metadata_df.to_csv(output_path, index=False)
            print(f"Created new metadata file at {output_path}")
        
        return metadata_df
        
    except Exception as e:
        print(f"An error occurred while generating the metadata file: {e}")
        return var_df  # Return original DataFrame if we encounter an error

In [35]:
dummy_data = "../data/dummy/dummy-data/asset_custdata.csv"

In [36]:
# Get file name from the argument passed
file_name = str.split(dummy_data, "/")[-1].split(".")[0]

In [37]:
#Generate report
var_df = generate_profiling_report(dummy_data)

Hashing sensitive column: CustomerBVN
Hashing sensitive column: ID Number


Render JSON: 100%|██████████| 1/1 [00:00<00:00, 32.66it/s]

Variables statistics have successfully been copied into asset_custdata_variables.csv file


In [38]:
var_df = var_df.T

In [40]:
var_df

,column_name,n_distinct,p_distinct,is_unique,n_unique,p_unique,type,hashable,value_counts_without_nan,value_counts_index_sorted,...,block_alias_counts,n_block_alias,block_alias_char_counts,script_counts,n_scripts,script_char_counts,category_alias_counts,n_category,category_alias_char_counts,word_counts
0,CustAID,216,1.0,True,216,1.0,Categorical,True,"{'bdd640fb-0667-4ad1-9c80-317fa3b1799d': 1, '1...","{'01d74256-3860-4ab6-96a4-02f23ae8cc93': 1, '0...",...,{'ASCII': 7776},1,"{'ASCII': {'-': 864, '4': 604, 'a': 487, '8': ...","{'Common': 5184, 'Latin': 2592}",2,"{'Common': {'-': 864, '4': 604, '8': 469, '9':...","{'Decimal Number': 4320, 'Lowercase Letter': 2...",3,"{'Dash_Punctuation': {'-': 864}, 'Decimal_Numb...","{'bdd640fb-0667-4ad1-9c80-317fa3b1799d': 1, '1..."
1,CustomerBVN,213,0.986111,False,212,0.981481,Categorical,True,{'3973e022e93220f9212c18d0d0c543ae7c309e46640d...,{'02003e343641bc47361af297a6adcf04bb9a0d36f75c...,...,{'ASCII': 13824},1,"{'ASCII': {'4': 899, '8': 891, 'd': 890, '5': ...","{'Common': 8620, 'Latin': 5204}",2,"{'Common': {'4': 899, '8': 891, '5': 887, '3':...","{'Decimal Number': 8620, 'Lowercase Letter': 5...",2,"{'Decimal_Number': {'4': 899, '8': 891, '5': 8...",{'3973e022e93220f9212c18d0d0c543ae7c309e46640d...
2,ID Number,189,0.875,False,164,0.759259,Categorical,True,{'a7937b64b8caa58f03721bb6bacf5c78cb235febe0e7...,{'0282d9b79f42c74c1550b20ff2dd16aafc3fe5d8ae9a...,...,{'ASCII': 13824},1,"{'ASCII': {'0': 967, '3': 910, 'c': 905, 'e': ...","{'Common': 8666, 'Latin': 5158}",2,"{'Common': {'0': 967, '3': 910, '7': 882, '8':...","{'Decimal Number': 8666, 'Lowercase Letter': 5...",2,"{'Decimal_Number': {'0': 967, '3': 910, '7': 8...",{'a7937b64b8caa58f03721bb6bacf5c78cb235febe0e7...
3,Gender,2,0.009259,False,0,0.0,Categorical,True,"{'Female': 123, 'Male': 93}","{'Female': 123, 'Male': 93}",...,{'ASCII': 1110},1,"{'ASCII': {'e': 339, 'a': 216, 'l': 216, 'F': ...",{'Latin': 1110},1,"{'Latin': {'e': 339, 'a': 216, 'l': 216, 'F': ...","{'Lowercase Letter': 894, 'Uppercase Letter': ...",2,"{'Lowercase_Letter': {'e': 339, 'a': 216, 'l':...","{'female': 123, 'male': 93}"
4,Occupation,187,0.865741,False,161,0.74537,Categorical,True,"{'Consulting civil engineer': 4, 'Pilot, airli...","{'Academic librarian': 1, 'Accommodation manag...",...,{'ASCII': 4366},1,"{'ASCII': {'e': 459, 'r': 384, 'i': 375, 'a': ...","{'Latin': 4000, 'Common': 366}",2,"{'Latin': {'e': 459, 'r': 384, 'i': 375, 'a': ...","{'Lowercase Letter': 3774, 'Space Separator': ...",6,"{'Lowercase_Letter': {'e': 459, 'r': 384, 'i':...","{'engineer': 25, 'manager': 18, 'officer': 15,..."
5,Account Officer,216,1.0,True,216,1.0,Categorical,True,"{'Carol Colon': 1, 'Michael Sloan': 1, 'Kristi...","{'Aaron Carlson': 1, 'Alison Williams': 1, 'Am...",...,{'ASCII': 2865},1,"{'ASCII': {'e': 260, 'a': 257, ' ': 228, 'n': ...","{'Latin': 2631, 'Common': 234}",2,"{'Latin': {'e': 260, 'a': 257, 'n': 219, 'r': ...","{'Lowercase Letter': 2182, 'Uppercase Letter':...",4,"{'Lowercase_Letter': {'e': 260, 'a': 257, 'n':...","{'michael': 8, 'matthew': 6, 'smith': 6, 'mr':..."


In [42]:
var_df.columns

Index(['column_name', 'n_distinct', 'p_distinct', 'is_unique', 'n_unique',
       'p_unique', 'type', 'hashable', 'value_counts_without_nan',
       'value_counts_index_sorted', 'ordering', 'n_missing', 'n', 'p_missing',
       'count', 'memory_size', 'imbalance', 'first_rows', 'chi_squared',
       'max_length', 'mean_length', 'median_length', 'min_length',
       'length_histogram', 'histogram_length', 'n_characters_distinct',
       'n_characters', 'character_counts', 'category_alias_values',
       'block_alias_values', 'block_alias_counts', 'n_block_alias',
       'block_alias_char_counts', 'script_counts', 'n_scripts',
       'script_char_counts', 'category_alias_counts', 'n_category',
       'category_alias_char_counts', 'word_counts'],
      dtype='object')

In [ ]:
var_df[['column_name', 'n', 'n_missing', 'p_missing', 'n_distinct', 'p_distinct', 'is_unique', 'n_unique',
       'p_unique', 'type', 'n_category', 'count','max_length', 'mean_length', 'min_length']]

,column_name,n_distinct,p_distinct,is_unique,n_unique,p_unique,type,n_missing,n,p_missing,count,max_length,mean_length,min_length,n_category
0,CustAID,216,1.0,True,216,1.0,Categorical,0,216,0.0,216,36,36.0,36,3
1,CustomerBVN,213,0.986111,False,212,0.981481,Categorical,0,216,0.0,216,64,64.0,64,2
2,ID Number,189,0.875,False,164,0.759259,Categorical,0,216,0.0,216,64,64.0,64,2
3,Gender,2,0.009259,False,0,0.0,Categorical,0,216,0.0,216,6,5.138889,4,2
4,Occupation,187,0.865741,False,161,0.74537,Categorical,0,216,0.0,216,57,20.212963,4,6
5,Account Officer,216,1.0,True,216,1.0,Categorical,0,216,0.0,216,25,13.263889,9,4


In [ ]:
    
print(f"Generated profiling report and saved to {file_name}.html.")